# 1D Gaussian Pulse Propagation with Absorbing Layers (PML)

Welcome! In this notebook, we will simulate the propagation of a Gaussian electromagnetic pulse through a dielectric medium with absorbing layers (Perfectly Matched Layers, or PMLs) using the `wakis` FDTD solver.

We will:

- Set up a 1D simulation domain with dielectric and absorbing regions
- Inject a Gaussian wavepacket as the source
- Analyze how different PML parameters affect absorption and reflection
- Visualize the results and compute metrics to quantify absorption

The animation below shows a typical result, where the pulse is absorbed by the PML:

![Gaussian pulse in PML](https://codimd.web.cern.ch/uploads/upload_377b25730e2b4ec04c803bbab82562da.gif)

## Simulation Setup

Let's begin by importing the required libraries and configuring the plotting environment.

In [ ]:
from scipy.constants import c as c_light
import numpy as np
from tqdm import tqdm

from wakis.solverFIT3D import SolverFIT3D
from wakis.gridFIT3D import GridFIT3D
from wakis.sources import WavePacket

# Plotting settings
import pyvista as pv
import matplotlib.animation
import matplotlib.pyplot as plt
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150
plt.ioff()

%matplotlib ipympl
pv.set_jupyter_backend('html')

## 1. Define the Simulation Domain and Absorbing Barriers

We define the size of the simulation box and set up a series of absorbing barriers (PMLs) with varying conductivity ($\sigma$).

- The domain is a rectangular box.
- The barriers are created as thin slabs at one end of the domain, each with increasing absorption strength.
- Each barrier is saved as an STL file for use in the simulation.

In [ ]:
# barrier dimensions
lx, ly, lz = 5., 5., 10.

# Domain bounds
xmin, xmax, ymin, ymax, zmin, zmax = -lx/2., lx/2., -ly/2., ly/2., -lz/2.,lz/2.

# Number of mesh cells
Nx = 4
Ny = 4
Nz = 800

# cell size
dx, dy, dz = lx/Nx, ly/Ny, lz/Nz

# Barriers
barrier = {}
stl_solids = {}
stl_materials = {}

n = 16
ss = np.linspace( np.sqrt(1.e-4), np.sqrt(1.e-1), n, )
ss = ss*ss

width = 2.*dz
pos_z_begin = zmax - n*width

for k, si in enumerate(ss):

    name = f'b{k}'
    file_name = f'notebooks/data/003_barrier{k}.stl'

    barrier[k] = pv.Cube( x_length=lx, y_length=ly, z_length=width, center=(0,0,pos_z_begin+k*width))
    barrier[k].save( file_name, )

    stl_solids[name] = file_name
    stl_materials[name] = [ 1.+si, 1., si, ]

In [ ]:
# Simulation grid
grid = GridFIT3D(xmin, xmax, ymin, ymax, zmin, zmax,
                 Nx, Ny, Nz,
                 stl_solids=stl_solids,
                 stl_materials=stl_materials)
for k in grid.stl_colors:
    grid.stl_colors[k] = 'tab:red'
grid.plot_solids(bounding_box=True)

## 2. Visualize the Simulation Grid

Let's plot the simulation grid and the positions of the absorbing barriers to verify our setup.

In [ ]:
# quick 3d plotting
solver.plot3D( 'Hy', add_stl='b1', clip_interactive=True, stl_opacity=0.4, cmap='gnuplot' )

## 3. Configure the FDTD Solver

We now set up the FDTD solver with the following parameters:

- **Boundary conditions**: Periodic in $x$ and $y$, PEC (perfect electric conductor) in $z$
- **Geometry import**: Enabled to use the STL barriers
- **Background material**: Vacuum

This prepares the solver for time-stepping.

In [ ]:
# boundary conditions
bc_low=['periodic', 'periodic', 'pec']
bc_high=['periodic', 'periodic', 'pec']

solver = SolverFIT3D(grid=grid,     # pass grid object
                     cfln=0.50,       # Default if no dt is defined
                     bc_low=bc_low,
                     bc_high=bc_high,
                     use_stl=True,     # Enables or disables geometry import
                     bg='vacuum',      # Background material
                     use_gpu=False,
                     )


## 4. Define and Visualize the Source

We inject a Gaussian wavepacket into the domain:

- The source is wide in $x$ and $y$, and narrow in $z$
- The temporal profile and spatial distribution are visualized below

In [ ]:
# Add Gaussian wavepacket
source = WavePacket(xs=slice(0,Nx), ys=slice(0,Ny),
                    sigmaxy=100.,
                    sigmaz=.4,
                    wavelength=16.*dz,
                    tinj=2.0,
                    amplitude=1.)

t = np.arange(0,(zmax-zmin)/c_light, solver.dt)

# Plot source in time
source.plot( t, )

# plot 2d
X, Y = np.meshgrid(solver.x, solver.y)
gaussxy = np.exp(-(X**2+Y**2)/(2*source.sigmaxy**2))
#plt.figure()
#plt.contourf(X,Y,gaussxy)

## 5. Time-Stepping Loop

We now run the main simulation loop:

- The fields are initialized to zero
- At each time step, the source is applied and the fields are updated
- We record the field values at the center of the domain for later analysis
- Optionally, you can enable 2D field plots during the simulation

In [ ]:
#initialize to 0
for d in ['x','y','z']:
    solver.E[:,:,:,d] = 0.
    solver.H[:,:,:,d] = 0.
    solver.J[:,:,:,d] = 0.

# Fields to save
Ex, Ey, Hx, Hy, Jz = [], [], [], [], []

plot2D=False #Turn on to generate 2d plots on-the-fly!

N1 = 5*int((2+zmax-zmin)/c_light/solver.dt)
for n in tqdm(range(N1)):

    #apply source
    source.update(solver, solver.dt*n)

    #Advance fields
    solver.one_step()

    if n%100 == 0:
        Ex.append(solver.E[Nx//2, Ny//2, :, 'x'])
        Ey.append(solver.E[Nx//2, Ny//2, :, 'y'])
        Hx.append(solver.H[Nx//2, Ny//2, :, 'x'])
        Hy.append(solver.H[Nx//2, Ny//2, :, 'y'])
        Jz.append(solver.J[Nx//2, Ny//2, :, 'z'])

    if plot2D and n%100 == 0:
        solver.plot2D(field='H', component='y', plane='ZY', pos=0.5, norm='symlog',
               vmin=-1, vmax=1, figsize=[8,4], cmap='RdBu', patch_alpha=0.1,
               add_patch=False, off_screen=True, n=n, interpolation='spline36',
               title='notebooks/data/003_Hy',)

## 6. Quick 3D Visualization

Let's quickly visualize the $H_y$ field in 3D, along with one of the absorbing barriers.

In [ ]:
# quick 3d plotting
solver.plot3D( 'Hy', add_stl='b1', clip_interactive=True, stl_opacity=0.4, cmap='gnuplot' )

In [ ]:
from argparse import Namespace as NS
fields = {
  'Ex': NS( values=np.array(Ex), units='', ),
  'Ey': NS( values=np.array(Ey), units='', ),
  'Hx': NS( values=np.array(Hx), units='', ),
  'Hy': NS( values=np.array(Hy), units='', ),
  'Jz': NS( values=np.array(Jz), units='', ),
}

# frames animation
fig, axs = plt.subplots( len(fields), 1, figsize=[8,8], dpi=150, )

axs[-1].set_xlabel(f'z [m]')
for ax, ( ki, vi, ) in zip(axs,fields.items()):

  ax.set_ylabel(f'Field {ki} {vi.units}')

  q = np.abs(vi.values).max()
  ax.set_ylim((-q,q))
  ax.set_xlim((zmin,zmax))

  # barriers
  for b, si in zip(barrier.values(),ss):
    xlo, xhi, ylo, yhi, zlo, zhi = b.bounds
    ax.axvspan(zlo, zhi, color='g', alpha=.1 + .6*si/ss.max())

  ls = [ ax.plot(solver.z, np.full_like( solver.z, np.nan, ), c='r')[0] for ax in axs ]

def animate(frame,ls,fields,axs):
  fig.gca()
  axs[0].set_title(f'frame {frame:6d}')
  # field
  for li, ( ki, vi, ) in zip(ls,fields.items()):
    li.set( data=(solver.z,vi.values[frame]), )
  return ls

anim = matplotlib.animation.FuncAnimation(fig, lambda frame: animate(frame,ls,fields,axs), frames=len(vi.values), )
anim.save(f'notebooks/data/003_movie.gif')

## 7. Animation of Field Evolution

We animate the evolution of the recorded field components as the pulse propagates and interacts with the absorbing layers.

- Each subplot shows a different field component
- The green shaded regions indicate the positions and strengths of the absorbing barriers

In [ ]:
from argparse import Namespace as NS
fields = {
  'Ex': NS( values=np.array(Ex), units='', ),
  'Ey': NS( values=np.array(Ey), units='', ),
  'Hx': NS( values=np.array(Hx), units='', ),
  'Hy': NS( values=np.array(Hy), units='', ),
  'Jz': NS( values=np.array(Jz), units='', ),
}

# frames animation
fig, axs = plt.subplots( len(fields), 1, figsize=[8,8], dpi=150, )

axs[-1].set_xlabel(f'z [m]')
for ax, ( ki, vi, ) in zip(axs,fields.items()):

  ax.set_ylabel(f'Field {ki} {vi.units}')

  q = np.abs(vi.values).max()
  ax.set_ylim((-q,q))
  ax.set_xlim((zmin,zmax))

  # barriers
  for b, si in zip(barrier.values(),ss):
    xlo, xhi, ylo, yhi, zlo, zhi = b.bounds
    ax.axvspan(zlo, zhi, color='g', alpha=.1 + .6*si/ss.max())

  ls = [ ax.plot(solver.z, np.full_like( solver.z, np.nan, ), c='r')[0] for ax in axs ]

def animate(frame,ls,fields,axs):
  fig.gca()
  axs[0].set_title(f'frame {frame:6d}')
  # field
  for li, ( ki, vi, ) in zip(ls,fields.items()):
    li.set( data=(solver.z,vi.values[frame]), )
  return ls

anim = matplotlib.animation.FuncAnimation(fig, lambda frame: animate(frame,ls,fields,axs), frames=len(vi.values), )
anim.save(f'notebooks/data/003_movie.gif')

## 8. Quantitative Metrics for Absorption

To quantify how well the PML absorbs the pulse, we define two metrics:

- **Area metric**: The sum of the absolute value of the field (measures total field energy)
- **Length metric**: The normalized arc length of the field profile (measures field oscillations)

We compute these metrics for each field component over time.

In [ ]:
def metric_area( f, ):
    return np.sum(np.abs(f))

def metric_length( f, ):
    df = np.diff(f)
    return np.sum(np.sqrt(1. + df*df))/len(df) - 1

for vi in fields.values():
    vi.area = np.array([ metric_area(fs) for fs in vi.values ])
    vi.area /= vi.area.max()
    vi.length = np.array([ metric_length(fs) for fs in vi.values ])
    vi.length /= vi.length.max()

## 9. Plot Absorption Metrics

Finally, we plot the evolution of the area and length metrics for each field component. This helps us assess the effectiveness of the absorbing layers.

- The area metric should decrease as the pulse is absorbed
- The length metric indicates how the field profile changes over time

In [ ]:
fig, ( ax1, ax2, ) = plt.subplots( 1, 2, tight_layout=True, )

for ki, vi in fields.items():
    ax1.plot( vi.area, label=ki, )
    ax2.plot( vi.length, label=ki, )

ax1.legend(), ax2.legend()